<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [319]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [321]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [248]:
# текст запроса
query_3_1 = f'''select count(id) as количество_вакансий
                from vacancies
'''

In [247]:
df = pd.read_sql_query(query_3_1, connection)
display(df)

,число_вакансий
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [249]:
query_3_2 = f'''select count(id) количество_работодателей
                from EMPLOYERS
'''

In [250]:
df = pd.read_sql_query(query_3_2, connection)
display(df)

,количество_работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [251]:
query_3_3 = f'''select count(id) количество_регионов
                from areas
'''

In [252]:
df = pd.read_sql_query(query_3_3, connection)
display(df)

,количество_регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [253]:
query_3_4 = f'''select count(id) сферы_деятельности
                from industries
'''

In [254]:
df = pd.read_sql_query(query_3_4, connection)
display(df)

,сферы_деятельности
0,294


***

В исследуемой базе данных имеется информация о 49 197 вакансиях, которые размещены 23 501 работодателем. В среднем, получается около 2 вакансий на каждого работодателя. 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [256]:
query_4_1 = f'''select count(v.id) as cnt, a.name as area
                from vacancies as v
                join areas as a on a.id = v.area_id
                group by a.name
                order by 1 desc
'''

In [257]:
df = pd.read_sql_query(query_4_1, connection)
display(df)

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
...,...,...
764,1,Тарко-Сале
765,1,Новоаннинский
766,1,Бирск
767,1,Сасово


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [278]:
query_4_2 = f'''select count(v.id)
                from vacancies as v
                where (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
                order by 1 desc
'''

In [279]:
df = pd.read_sql_query(query_4_2, connection)
display(df)

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [236]:
query_4_3 = f'''select 
                    round(avg(v.salary_from)) as avg_salary_from,
                    round(avg(v.salary_to)) as avg_salary_to
                from vacancies as v
            
'''

In [237]:
df = pd.read_sql_query(query_4_3, connection)
display(df)

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [260]:
query_4_5 = f'''select
                    count(id),
                    v.schedule,
                    v.employment
                from vacancies as v
                group by v.schedule, v.employment
                order by count(id) desc  
                limit 3        
'''

In [261]:
df = pd.read_sql_query(query_4_5, connection)
display(df)

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [24]:
query_4_6 = f'''select count(id), v.experience
                from vacancies as v
                group by v.experience
                order by count(id)        
'''

In [25]:
df = pd.read_sql_query(query_4_6, connection)
display(df)

,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

Наиболее "популярным" у работодателей является график работы "Полный день" и "Полная занятость". Чаще всего на работу требуются сотрудники с опытом работы от 1 года до 6 лет - т.е. большинство работодателей не заинтересованы в обучении сотрудников. Большинство вакансий размещены в Москве, Санкт-Петербурге и столицах соседних государств - Белоруссии и Казахстана. Средние значения зарплатной вилки нельзя считать достоверными - заработные платы в групных городах отличаются от зарплат в других регионах страны, к тому же у более половины вакансий не заполнены данные о верхней или нижней границе заработной платы или не заполнены данные о зараплате совсем.  

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [280]:
query_5_1 = f'''select count(v.id), e.name
                from vacancies as v
                join employers as e on e.id = v.employer_id
                group by 2
                order by 1 desc
                limit 5
'''

In [281]:
df = pd.read_sql_query(query_5_1, connection)
display(df)

,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [282]:
query_5_2 = f'''select 
                    a.name,
                    count(distinct e.id) as cnt_emp
                from areas as a
                left join employers as e on e.area = a.id
                left join vacancies as v on a.id = v.area_id 
                where v.id is null
                group by a.name
                order by count(e.id) desc
                limit 1
               
'''

In [283]:
df = pd.read_sql_query(query_5_2, connection)
display(df)

,name,cnt_emp
0,Россия,410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [284]:
query_5_3 = f'''select 
                   e.name,
                   count(distinct a.id)
                from vacancies v
                left join employers as e on v.employer_id = e.id
                left join areas as a on a.id = v.area_id 
                group by e.name
                order by 2 desc
            
               
'''

In [285]:
df = pd.read_sql_query(query_5_3, connection)
display(df)

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [286]:
query_5_4 = f'''select 
                   count(*)
                from employers as e 
                left join employers_industries as i on e.id = i.employer_id
                where i.industry_id is null              
'''

In [287]:
df = pd.read_sql_query(query_5_4, connection)
display(df)

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [290]:
query_5_5 = f'''select 
                   e.name
                from employers as e 
                left join employers_industries as i on e.id = i.employer_id
                group by e.name
                having count(i.industry_id)=4
                order by e.name
                offset 2
                limit 1      
'''

In [291]:
df = pd.read_sql_query(query_5_5, connection)
display(df)

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [299]:
query_5_6 = f'''select 
                   count(e.id)
                from employers as e 
                left join employers_industries as ei on e.id = ei.employer_id
                left join industries as i on ei.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'
'''

In [300]:
df = pd.read_sql_query(query_5_6, connection)
display(df)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [294]:
import requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
##print(page.prettify())
anch = page.find("span",id="Список_городов-миллионеров")
table = anch.find_next('table')
rows = table.find_all('tr')
cities = []
for row in rows:
    cities.append(row.find("a").text)
cities.pop(0)
city = tuple(cities)
print(city)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [295]:
query_5_7 = f'''select 
                   a.name, count(v.id)
                from vacancies v
                left join employers as e on v.employer_id = e.id
                left join areas as a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name IN {city}
                group by a.name
                
                UNION ALL
                
                select
                    'total', count(v.id)
                from vacancies v
                left join employers as e on v.employer_id = e.id
                left join areas as a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name IN {city}
                order by 2
               
'''

In [296]:
df = pd.read_sql_query(query_5_7, connection)
display(df)

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

Больше всего вакансий публикует "Яндекс". Вакансии публикуются в разных регионах и разных сферах деятельности. Это может быть связано как с развитием различных направлений компании так и с большой текучкой кадров.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [181]:
query_6_1 = f'''select 
                   count(*)
                from vacancies as v
                where lower(v.name) LIKE '%data%' OR lower(v.name) LIKE '%данн%'
                
'''

In [182]:
df = pd.read_sql_query(query_6_1, connection)
display(df)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [128]:
query_6_2 = f'''select 
                   count(*)
                from vacancies as v
                where (v.name ILIKE '%data scientist%' OR
                      v.name ILIKE '%data science%' OR
                      v.name ILIKE '%исследователь данных%' OR
                      (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
                      v.name ILIKE '%machine learning%' OR
                      v.name ILIKE '%машинн%обучен%') 
                      AND
                      (v.name ILIKE '%junior%' OR
                       v.experience ILIKE '%Нет опыта%' OR
                       v.employment ILIKE '%Стажировка%')
                
'''

In [129]:
df = pd.read_sql_query(query_6_2, connection)
display(df)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [301]:
query_6_3 = f'''select 
                   count(*)
                from vacancies as v
                where (v.name  ILIKE '%data scientist%' OR
                      v.name ILIKE '%data science%' OR
                      v.name ILIKE '%исследователь данных%' OR
                      (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
                      v.name ILIKE '%machine learning%' OR
                      v.name ILIKE '%машинн%обучен%')
                      AND
                     (v.key_skills LIKE '%SQL%' or v.key_skills LIKE '%postgres%')
                
'''

In [302]:
df = pd.read_sql_query(query_6_3, connection)
display(df)

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [303]:
query_6_4 = f'''select 
                   count(*)
                from vacancies as v
                where ((v.name  ILIKE '%data scientist%') OR
                      (v.name ILIKE '%data science%') OR
                      (v.name ILIKE '%исследователь данных%') OR
                      (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
                      (v.name ILIKE '%machine learning%') OR
                      (v.name ILIKE '%машинн%обучен%'))
                      AND
                     v.key_skills LIKE '%Python%'
                
'''

In [304]:
df = pd.read_sql_query(query_6_4, connection)
display(df)

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [305]:

query_6_5 = f'''select 
                   round(avg(LENGTH(v.key_skills)-LENGTH(replace(v.key_skills, CHR(9), ''))+1),2) as среднее_число_навыков
                from vacancies as v
                where (v.name  ILIKE '%data scientist%' OR
                      v.name ILIKE '%data science%' OR
                      v.name ILIKE '%исследователь данных%' OR
                      (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
                      v.name ILIKE '%machine learning%' OR
                      v.name ILIKE '%машинн%обучен%')
                      AND 
                      v.key_skills IS NOT NULL
                
                
'''

In [306]:
df = pd.read_sql_query(query_6_5, connection)
display(df)

,среднее_число_навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [307]:
query_6_6 = f'''select 
                   v.experience, round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)))
                from vacancies as v
                where (v.name  ILIKE '%data scientist%' OR
                      v.name ILIKE '%data science%' OR
                      v.name ILIKE '%исследователь данных%' OR
                      (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
                      v.name ILIKE '%machine learning%' OR
                      v.name ILIKE '%машинн%обучен%')
                      AND 
                      (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
                  group by v.experience
                
                
'''

In [308]:
df = pd.read_sql_query(query_6_6, connection)
display(df)

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

# Общий вывод по проекту

Среднее число ключевых навыков в вакансиях для начинающих специалистов DS

In [309]:
query_6_7 = f'''select 
                   round(avg(LENGTH(v.key_skills)-LENGTH(replace(v.key_skills, CHR(9), ''))+1),2) as среднее_число_навыков
                from vacancies as v
                where (v.name  ILIKE '%data scientist%' OR
                      v.name ILIKE '%data science%' OR
                      v.name ILIKE '%исследователь данных%' OR
                      (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
                      v.name ILIKE '%machine learning%' OR
                      v.name ILIKE '%машинн%обучен%')
                        AND
                      (v.name ILIKE '%junior%' OR
                       v.experience ILIKE '%Нет опыта%' OR
                       v.employment ILIKE '%Стажировка%')
                      AND 
                      v.key_skills IS NOT NULL
                
                
'''

In [310]:
df = pd.read_sql_query(query_6_7, connection)
display(df)

,среднее_число_навыков
0,7.27


Средняя зарплата для начинающего специалиста DS

In [312]:
query_6_8 = f'''select 
                   v.experience, round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)))
                from vacancies as v
                where (v.name  ILIKE '%data scientist%' OR
                      v.name ILIKE '%data science%' OR
                      v.name ILIKE '%исследователь данных%' OR
                      (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
                      v.name ILIKE '%machine learning%' OR
                      v.name ILIKE '%машинн%обучен%')
                           AND
                      (v.name ILIKE '%junior%' OR
                       v.experience ILIKE '%Нет опыта%' OR
                       v.employment ILIKE '%Стажировка%')
                      AND 
                      (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
                  group by v.experience
                
                
'''

In [313]:
df = pd.read_sql_query(query_6_8, connection)
display(df)

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,55000.0


В каких городах размещены вакансии, подходящие для начинающего специалиста в области DS

In [317]:
query_6_9 = f'''select 
                   a.name,
                   count(distinct v.id)
                from vacancies v
                left join employers as e on v.employer_id = e.id
                left join areas as a on a.id = v.area_id 
                   where (v.name  ILIKE '%data scientist%' OR
                      v.name ILIKE '%data science%' OR
                      v.name ILIKE '%исследователь данных%' OR
                      (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
                      v.name ILIKE '%machine learning%' OR
                      v.name ILIKE '%машинн%обучен%')
                           AND
                      (v.name ILIKE '%junior%' OR
                       v.experience ILIKE '%Нет опыта%' OR
                       v.employment ILIKE '%Стажировка%')
                group by a.name
                order by 2 desc
            
               
'''

In [322]:
df = pd.read_sql_query(query_6_9, connection)
display(df)

,name,count
0,Москва,19
1,Санкт-Петербург,9
2,Новосибирск,5
3,Минск,2
4,Воронеж,2
5,Екатеринбург,2
6,Ростов-на-Дону,2
7,Пермь,2
8,Томск,2
9,Казань,2


Начинающему специалисту в области DS работу проще всего найти в крупных городах. В маленьких городах если и необходимы специалисты такого профиля, то либо нужны сотрудники с опытом, либо вакансии не размещают, а ищут сотрудника по рекомендациям.

К начинающему специалисту в области DS предъявляются более высокие требования (ключевые навыки).

In [316]:
connection.close()